In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


In [9]:

class PredictValues:
    def __init__(self, years, revenue, net_income):
        self.years = years
        self.revenue = revenue
        self.net_income = net_income
        self.feature_year_arr = []
        self.feature_revenue_arr = []
        self.feature_net_income_arr = []
        self.n_steps = 3
        self.n_features = 1
        self.model = self.build_model()
        self.train_model()

    def prepare_data(self):
        X, y_revenue, y_net_income = [], [], []
        for i in range(len(self.revenue) - self.n_steps):
            X.append(self.revenue[i:i + self.n_steps])
            y_revenue.append(self.revenue[i + self.n_steps])
            y_net_income.append(self.net_income[i + self.n_steps])
        X = np.array(X).reshape((len(X), self.n_steps, self.n_features))
        y_revenue = np.array(y_revenue)
        y_net_income = np.array(y_net_income)
        return X, y_revenue, y_net_income

    def build_model(self):
        inputs = Input(shape=(self.n_steps, self.n_features))
        lstm_out = LSTM(50, activation='relu')(inputs)
        revenue_output = Dense(1, name='revenue_output')(lstm_out)
        net_income_output = Dense(1, name='net_income_output')(lstm_out)
        model = Model(inputs=inputs, outputs=[revenue_output, net_income_output])
        model.compile(optimizer='adam', loss='mse')
        return model

    def train_model(self):
        X, y_revenue, y_net_income = self.prepare_data()
        self.model.fit(X, {'revenue_output': y_revenue, 'net_income_output': y_net_income}, epochs=300, verbose=1)

    def predict_future(self, input_seq, n_years):
        current_input = np.array(input_seq).reshape((1, self.n_steps, self.n_features))
        current_year = self.years[-1]
        for _ in range(n_years):
            pred_revenue, pred_net_income = self.model.predict(current_input, verbose=0)
            self.feature_year_arr.append(current_year)
            self.feature_net_income_arr.append(round(pred_net_income[0][0], 2))
            self.feature_revenue_arr.append(round(pred_revenue[0][0], 2))
            # predictions.append((pred_revenue[0][0], pred_net_income[0][0]))
            # Reshape the predicted revenue to match the dimensions: (1, 1, n_feature)
            new_step = pred_revenue.reshape((1, 1, self.n_features))
            # Remove the oldest time step and append the new prediction
            current_input = np.concatenate((current_input[:, 1:, :], new_step), axis=1)
            current_year += 1
        return {'years': self.feature_year_arr, 'revenue': self.feature_revenue_arr, 'net_income': self.feature_net_income_arr}


In [10]:
years = [year for year in range(2015, 2025)]
revenue = [110, 125, 133, 146, 158, 172, 187, 196, 210, 230]
net_income = [10, 12, 13, 15, 16, 17, 19, 20, 22, 24]

In [11]:
input_seq = revenue[-3:]  # Last 3 revenue values as input sequence
pv = PredictValues(years, revenue, net_income)
future_predictions = pv.predict_future(input_seq, 5)

Epoch 1/300
1/1 [==============================] - 5s 5s/step - loss: 34962.1328 - revenue_output_loss: 34116.7461 - net_income_output_loss: 845.3849
Epoch 2/300
1/1 [==============================] - 0s 16ms/step - loss: 34668.1602 - revenue_output_loss: 33869.6836 - net_income_output_loss: 798.4774
Epoch 3/300
1/1 [==============================] - 0s 17ms/step - loss: 34365.6484 - revenue_output_loss: 33611.3633 - net_income_output_loss: 754.2867
Epoch 4/300
1/1 [==============================] - 0s 15ms/step - loss: 34055.4961 - revenue_output_loss: 33341.0742 - net_income_output_loss: 714.4213
Epoch 5/300
1/1 [==============================] - 0s 19ms/step - loss: 33736.0781 - revenue_output_loss: 33055.8750 - net_income_output_loss: 680.2016
Epoch 6/300
1/1 [==============================] - 0s 22ms/step - loss: 33403.2500 - revenue_output_loss: 32750.4551 - net_income_output_loss: 652.7952
Epoch 7/300
1/1 [==============================] - 0s 19ms/step - loss: 33050.3516 - reven

In [12]:
for val in future_predictions:
    print(f'{val} : {future_predictions[val]}')

years : [2024, 2025, 2026, 2027, 2028]
revenue : [241.7, 258.29, 277.32, 293.76, 313.03]
net_income : [24.94, 26.33, 28.49, 30.11, 31.98]
